In [5]:
content=`cat app.py` yq eval '.$graph[1].requirements.InitialWorkDirRequirement.listing[0].entry=strenv(content)' workflow.cwl.template > workflow.cwl


In [6]:
cat workflow.cwl

cwlVersion: v1.0
$namespaces:
  s: https://schema.org/
s:softwareVersion: 1.0.0
schemas:
  - http://schema.org/version/9.0/schemaorg-current-http.rdf
$graph:
  - class: Workflow
    id: main
    label: Water bodies detection based on NDWI and the otsu threshold
    doc: Water bodies detection based on NDWI and otsu threshold applied to a single Sentinel-2 COG STAC item
    requirements: []
    inputs:
      aoi:
        label: area of interest
        doc: area of interest as a bounding box
        type: string
      epsg:
        label: EPSG code
        doc: EPSG code
        type: string
        default: "EPSG:4326"
      bands:
        label: bands used for the NDWI
        doc: bands used for the NDWI
        type: string[]


        default: ["green", "nir"]
      item:
        doc: Reference to a STAC item
        label: STAC item reference
        type: string
    outputs:
      - id: stac_catalog
        outputSource:
          - node_detect/stac-catalog
        type: Directory
    steps:
      node_detect:
        run: "#detect-water-body"
        in:
          item: item
          aoi: aoi
          epsg: epsg
          band: bands
        out:
          - stac-catalog
  - class: CommandLineTool
    id: detect-water-body
    requirements:
      InlineJavascriptRequirement: {}
      EnvVarRequirement:
        envDef: {}
      ResourceRequirement:
        coresMax: 1
        ramMax: 512
      InitialWorkDirRequirement:
        listing:
          - entryname: app.py
            entry: |-
              import os
              import click
              import pystac
              import rasterio
              from skimage.filters import threshold_otsu
              from rasterio.mask import mask
         

In [3]:
cat params.yaml

item: "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A"
aoi: "-121.399,39.834,-120.74,40.472"
epsg: "EPSG:4326"


In [4]:
cwltool workflow.cwl params.yaml


INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved 'workflow.cwl' to 'file:///workspace/quickwin-inline-code/workflow.cwl'
INFO [workflow ] start
INFO [workflow ] starting step node_detect
INFO [step node_detect] start
WARN[0000] "/" is not a shared mount, this could cause issues or missing mounts with rootless containers 
Error: error inspecting object: no such object: "qw:latest"
INFO ['podman', 'pull', 'qw:latest']
Error: short-name "qw:latest" did not resolve to an alias and no unqualified-search registries are defined in "/etc/containers/registries.conf"
ERROR Workflow error, try again with --debug for more information:
Docker is not available for this tool, try --no-container to disable Docker, or install a user space Docker replacement like uDocker with --user-space-docker-cmd.: Command '['podman', 'pull', 'qw:latest']' returned non-zero exit status 125.


: 1

In [2]:
podman build -t registry:5000/qw:latest .


STEP 1/4: FROM docker.io/python:slim-bullseye
Trying to pull docker.io/library/python:slim-bullseye...
Getting image source signatures
Copying blob 226f2f9a8120 [================================>---] 2.6MiB / 2.9MiB
Copying blob 29ff364e54a5 [====================================] 1.0MiB / 1.0MiB
Copying blob bfe30d23a9fb [====================================] 244.0b / 244.0b
Copying blob c0edef2937fa [==>--------------------------------] 2.3MiB / 30.0MiB
Copying blob 4e7818858816 [=====>-----------------------------] 1.8MiB / 10.7MiB
Copying blob 226f2f9a8120 [====================================] 2.9MiB / 2.9MiB
Copying blob 29ff364e54a5 done  
Copying blob bfe30d23a9fb done  
Copying blob c0edef2937fa [====>------------------------------] 4.4MiB / 30.0MiB
Copying blob 4e7818858816 [=============>---------------------] 4.2MiB / 10.7MiB
Copying blob 226f2f9a8120 done  
Copying blob 29ff364e54a5 done  
Copying blob bfe30d23a9fb done  
Copying blob c0edef2937fa [==========>--------------

In [8]:
s="detect-water-body" t="registry:5000/qw:latest" yq -i eval '(.$graph[] | select (.id == env(s)) ).hints.DockerRequirement.dockerPull = env(t)' workflow.cwl

In [9]:
cwltool workflow.cwl params.yaml

INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved 'workflow.cwl' to 'file:///workspace/quickwin-inline-code/workflow.cwl'
INFO [workflow ] start
INFO [workflow ] starting step node_detect
INFO [step node_detect] start
WARNING [job node_detect] Skipping Docker software container '--memory' limit despite presence of ResourceRequirement with ramMin and/or ramMax setting. Consider running with --strict-memory-limit for increased portability assurance.
WARNING [job node_detect] Skipping Docker software container '--cpus' limit despite presence of ResourceRequirement with coresMin and/or coresMax setting. Consider running with --strict-cpu-limit for increased portability assurance.
INFO [job node_detect] /tmp/q0dc9yel$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/q0dc9yel,target=/MRdBdC \
    --mount=type=bind,source=/tmp/qezkygg2,target=/tmp \
    --workdir=/MRdBdC \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --cidfile=/tmp

In [8]:
tree -l /workspace/quickwin-inline-code/3edn2u1q

/workspace/quickwin-inline-code/3edn2u1q
├── S2B_10TFK_20210713_0_L2A
│   ├── S2B_10TFK_20210713_0_L2A.json
│   └── otsu.tif
├── __pycache__
│   └── app.cpython-312.pyc
├── app.py
└── catalog.json

2 directories, 5 files


In [10]:
podman push registry:5000/qw:latest

Getting image source signatures
WARN[0000] failed, retrying in 1s ... (1/3). Error: trying to reuse blob sha256:3c8879ab2cf25eb049af95711715b5b99d97490ab4eedbe8d0f160f53ef5a645 at destination: pinging container registry registry:5000: Get "http://registry:5000/v2/": dial tcp 10.46.78.158:5000: connect: connection refused 
Getting image source signatures
WARN[0001] failed, retrying in 1s ... (2/3). Error: trying to reuse blob sha256:3c8879ab2cf25eb049af95711715b5b99d97490ab4eedbe8d0f160f53ef5a645 at destination: pinging container registry registry:5000: Get "http://registry:5000/v2/": dial tcp 10.46.78.158:5000: connect: connection refused 
Getting image source signatures
WARN[0002] failed, retrying in 1s ... (3/3). Error: trying to reuse blob sha256:3c8879ab2cf25eb049af95711715b5b99d97490ab4eedbe8d0f160f53ef5a645 at destination: pinging container registry registry:5000: Get "http://registry:5000/v2/": dial tcp 10.46.78.158:5000: connect: connection refused 
Getting image source signatu

: 125

In [ ]:
/opt/conda/bin/calrissian --pod-nodeselectors /etc/calrissian/pod-node-selector.yml --stdout /calrissian/results.json --max-ram 16G --max-cores "8" --tmp-outdir-prefix /calrissian/tmp/ --outdir /calrissian/